Q1

In [1]:
1

1

In [2]:
import pandas as pd 

In [3]:
df = pd.read_csv('T124OPPE2_Preprocessing_V1.csv')
df.shape

(4000, 11)

Q2

In [4]:
df['Gender'].unique()

array(['Female', 'Male', 'Unknown'], dtype=object)

In [5]:
df[df['Gender'] == 'Unknown'].shape

(7, 11)

Q3

In [6]:
df[df['Age']<0].shape

(8, 11)

Q4

In [7]:
df['GlucoseLevel'].isnull().sum()

0

In [8]:
df[df['GlucoseLevel'] < 0].shape

(9, 11)

Q5

In [9]:
df['LivesIn'].unique()

array(['City', 'Village', 'Unknown'], dtype=object)

In [10]:
df[df['LivesIn'] == 'Unknown'].shape

(5, 11)

Q6

In [11]:
df['BMI'].isnull().sum()

149

In [12]:
df[df['BMI'] < 0].shape

(0, 11)

Q7

In [13]:
df['SmokingStatus'].unique()

array(['never smoked', 'smokes', 'Unknown', 'formerly smoked'],
      dtype=object)

In [14]:
df[df['SmokingStatus'] == 'Unknown'].shape

(1204, 11)

Q8

In [15]:
import numpy as np 
np.array([2, 3, 1, np.nan]).mean()

nan

In [36]:
df[df['BMI'].notna()]['BMI'].mean()

28.857958971695663

Q9

In [17]:
for column in df.select_dtypes(include='object'):
    print(column)
    print(df[column].unique(),'\n')

Gender
['Female' 'Male' 'Unknown'] 

HasTension
['Yes' 'No'] 

AnyHeartDisease
['No' 'Yes'] 

NeverMarried
['Yes' 'No'] 

Occupation
['Self-employed' 'Private' 'children' 'Govt_job' 'Never_worked'] 

LivesIn
['City' 'Village' 'Unknown'] 

SmokingStatus
['never smoked' 'smokes' 'Unknown' 'formerly smoked'] 

HeartAttack
['No' 'Yes'] 



In [18]:
df[
    (df['LivesIn'] == 'City') &
    ((df['SmokingStatus'] == 'formerly smoked') | (df['SmokingStatus'] == 'smokes')) &
    (df['HeartAttack'] == 'Yes')
].shape

(52, 11)

Q10

In [19]:
# female patients without tension, without any heart disease and never married
print(df[
    (df['Gender'] == 'Female') & 
    (df['HasTension'] == 'No') &
    (df['AnyHeartDisease'] == 'No') &
    (df['NeverMarried'] == 'Yes')
].shape[0])

# female patients without tension, without any heart disease and either currently married or married before
print(df[
    (df['Gender'] == 'Female') & 
    (df['HasTension'] == 'No') &
    (df['AnyHeartDisease'] == 'No') &
    (df['NeverMarried'] == 'No')
].shape[0])

# male patients without tension, without any heart disease and never married
print(df[
    (df['Gender'] == 'Male') & 
    (df['HasTension'] == 'No') &
    (df['AnyHeartDisease'] == 'No') &
    (df['NeverMarried'] == 'Yes')
].shape[0])

# male patients with tension, with a heart disease and never married
print(df[
    (df['Gender'] == 'Male') & 
    (df['HasTension'] == 'Yes') &
    (df['AnyHeartDisease'] == 'Yes') &
    (df['NeverMarried'] == 'Yes')
].shape[0])

1335
754
795
24


Q11

In [20]:
for column in df.select_dtypes(include='object'):
    print(column, end=' | ')

Gender | HasTension | AnyHeartDisease | NeverMarried | Occupation | LivesIn | SmokingStatus | HeartAttack | 

Q12

In [21]:
df['HeartAttack'].value_counts()

HeartAttack
No     3806
Yes     194
Name: count, dtype: int64

Q13

In [22]:
X = df.copy()
y = X['HeartAttack']
X.drop('HeartAttack', inplace=True, axis=1)

In [23]:
X.shape, y.shape

((4000, 10), (4000,))

In [24]:
# Mapping Yes-1 and 0-No 
def mapper(i):
    if i == "Yes":
        return 1 
    else:
        return 0 

y = y.map(mapper)

In [25]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.3, stratify=y)

In [26]:
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer 
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, MinMaxScaler, StandardScaler

gender_pipe = make_pipeline(
    SimpleImputer(strategy='most_frequent'),
    OrdinalEncoder()
)

age_pipe = make_pipeline(
    SimpleImputer(strategy='mean'),
    StandardScaler()
)
tension_heart_disease_never_married = make_pipeline(
    OrdinalEncoder()
)
occupation_pipe = make_pipeline(
    OneHotEncoder(sparse_output=False)
)
lives_in_pipe = make_pipeline(
    SimpleImputer(strategy='most_frequent'),
    OrdinalEncoder()
)
glucose_pipe = make_pipeline(
    SimpleImputer(strategy='mean'),
    MinMaxScaler()
)
bmi_pipe = make_pipeline(
    SimpleImputer(strategy='mean'),
    StandardScaler()
)
smoking_pipe = make_pipeline(
    SimpleImputer(strategy='most_frequent'),
    OneHotEncoder(sparse_output=False)
)
pipe = make_column_transformer(
    (gender_pipe, ['Gender']),
    (age_pipe, ['Age']),
    (tension_heart_disease_never_married, ['HasTension', 'AnyHeartDisease', 'NeverMarried']),
    (occupation_pipe, ['Occupation']),
    (lives_in_pipe, ['LivesIn']),
    (glucose_pipe, ['GlucoseLevel']),
    (bmi_pipe, ['BMI']),
    (smoking_pipe, ['SmokingStatus']),
)

pipe.set_output(transform='pandas')

ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('ordinalencoder',
                                                  OrdinalEncoder())]),
                                 ['Gender']),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 ['Age']),
                                ('pipeline-3',
                                 Pipeline(steps=[('ordinalencoder',
                                                  OrdinalEncoder())]),
                                 [...
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('minmaxscaler',
                                                  MinMaxScaler())]),
                                 ['GlucoseLevel']),
                                ('pipeline-7',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 ['BMI']),
                                ('pipeline-8',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(sparse_output=False))]),
                                 ['SmokingStatus'])])

In [27]:
pipe.fit_transform(X_train)

,pipeline-1__Gender,pipeline-2__Age,pipeline-3__HasTension,pipeline-3__AnyHeartDisease,pipeline-3__NeverMarried,pipeline-4__Occupation_Govt_job,pipeline-4__Occupation_Never_worked,pipeline-4__Occupation_Private,pipeline-4__Occupation_Self-employed,pipeline-4__Occupation_children,pipeline-5__LivesIn,pipeline-6__GlucoseLevel,pipeline-7__BMI,pipeline-8__SmokingStatus_Unknown,pipeline-8__SmokingStatus_formerly smoked,pipeline-8__SmokingStatus_never smoked,pipeline-8__SmokingStatus_smokes
2947,0.0,0.436336,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,2.0,0.366568,0.284421,1.0,0.0,0.0,0.0
2234,1.0,-1.827889,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.511861,-1.186687,1.0,0.0,0.0,0.0
2013,1.0,-1.150738,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.464789,0.065045,0.0,0.0,1.0,0.0
2661,1.0,-0.798054,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,2.0,0.349889,0.929643,1.0,0.0,0.0,0.0
2275,0.0,0.744934,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.364344,-0.412420,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2990,0.0,0.744934,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,2.0,0.322090,-0.451133,0.0,0.0,1.0,0.0
2171,0.0,-0.489457,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,2.0,0.485915,-0.515655,0.0,0.0,0.0,1.0
2578,0.0,0.965361,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.313566,-1.147973,0.0,1.0,0.0,0.0
1328,1.0,-0.357201,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.325056,0.232803,0.0,0.0,1.0,0.0


Gender: Impute with most frequent then ordinally encode.

Age: Impute with mean then standard scale.

HasTension: Ordinally encode.

AnyHeartDisease:Ordinally encode.

NeverMarried:Ordinally encode.

Occupation: One hot encode.

LivesIn: Impute with most frequent then ordinally encode.

GlucoseLevel: Impute with mean, then min-max scaling.

BMI: Impute with mean, then standard scale.

SmokingStatus: Impute with most frequent, then one hot encode